# Summary
### Managing files stored on AWS S3 with Boto3

<p>
This project uses the Simple Notification Service (SNS) of AWS. Several basic functions like creating topics, subscriptions, publishing are explained. The functionality is extended with a newly created helper package by me. This is a work in progress. 
</p>

<p> 
It relies heavily on the  
<a href = https://boto3.amazonaws.com/v1/documentation/api/latest/index.html target=_blank> 
Boto3 documentation.</a> <br> 
According to the doc: 
</p> 

<p> 
“You use the AWS SDK for Python (Boto3) to create, configure, and manage AWS services, such as Amazon Elastic Compute Cloud (Amazon EC2) and Amazon Simple Storage Service (Amazon S3). The SDK provides an object-oriented API as well as low-level access to AWS services.” 
</p> 

<p>This project creates an  

### S3_helpers_pckg 

<p> 
The package stores a class with useful helper functions, mostly manipulating the dicts of responses.<br> 
The functions are mostly self defined, but other functions for example from Github and the doc are integrated.<br> 
In this case credits are given.<br> 
The pckg is a work in progress. <br>
Functions defined here are added later to the helper package.

</p> 

<p>Several topics are examined here. <br> 
For example:</p> 
<ul> 
<li>Setting up AWS for SNS.</li> 
<li>Creating a client.</li> 
<li>Creating a topic.</li> 
<li>Subscriptions.</li> 
<li>Unsubscribe.</li> 
<li>Endpoints.</li> 
<li>Publishing messages.</li>     
</ul> 


<p> 
The credentials are secured with a <a href="www.dotenv.org/docs" target=_blank> 
dotenv.</a>
</p> 
 

# Import packages

In [3]:
# Import packages

import pandas as pd
import pandasql
from pandasql import sqldf

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import numpy as np
import random
import os as os
import time
import pprint
import sys
import re
import json
import glob 
import jinja2 # for styling data frames 
from IPython.display import display, HTML # Displaying HTML in Jupyter notebook

from pathlib import *

# security
from dotenv import load_dotenv
import logging

# display html in code cells
from IPython.display import HTML, display, Markdown, Latex, Image

#### Import Boto

In [4]:
# import boto3
import boto3
import botocore

### S3_helpers_pckg

<p>
stores a class with useful helper functions, mostly manipulating the dicts of responses.<br>
The functions are mostly self defined, but other functions for example from Github and the doc are integrated.<br>
In this case credits are given.<br>
The pckg is a work in progress.
</p>

In [5]:
import S3_helpers_pckg

In [6]:
initpy_path = S3_helpers_pckg

print(type(initpy_path))
print(str(initpy_path)[1:30])


<class 'module'>
module 'S3_helpers_pckg' from


In [7]:
from S3_helpers_pckg import S3_helpers

### Checkout the directories

In [8]:
cwd = os.getcwd()
# print(cwd)

In [9]:
# os.chdir('G:\Other computers\Mein Laptop (1)\data_camp_projects\Turing_DataAnalysis')
os.chdir('G:\Other computers\Mein Laptop (1)\data_camp_projects\AWS_boto3')

In [10]:
### List directories

directories_list = os.listdir()
directories_list

['AWS_boto3_S3_FileManagement.ipynb',
 'txt',
 '.ipynb_checkpoints',
 'data',
 '.env',
 'S3_helpers_pckg',
 'upload_files',
 'download_files',
 'csv',
 'html',
 '.git',
 'README.md',
 '.gitignore']

In [11]:
file_list = glob.glob("*")
print(file_list)

['AWS_boto3_S3_FileManagement.ipynb', 'txt', 'data', 'S3_helpers_pckg', 'upload_files', 'download_files', 'csv', 'html', 'README.md']


### Creating requirement files

In [12]:
print("Current version of Python is ", sys.version)
print(pd.__version__)
print(np.__version__)
print(sns.__version__)

Current version of Python is  3.10.4 | packaged by conda-forge | (main, Mar 30 2022, 08:38:02) [MSC v.1916 64 bit (AMD64)]
1.4.3
1.21.5
0.11.2


#### Making directories

In [13]:
if not os.path.exists("txt"):
    # if the demo_folder directory is not present 
    # then create it.
    os.makedirs("txt")
    
if not os.path.exists("upload_files"):
    os.makedirs("upload_files")
    
if not os.path.exists("data"):
    os.makedirs("data")


In [14]:
!conda list > txt/requirements_file_conda_boto3.txt
!pip list > txt/requirements_file_pip_boto3.txt

# Setting up AWS resources

## Import AWS keys


In [15]:
 %run S3_helpers_pckg/settings.py

In [16]:
# print(Secret_Access_Key)
# print(Access_Key_ID)

## AWS Simple Notificaton Service with Boto 3
<p>
is a fully managed Pub/Sub service for A2A and A2P messaging.<br>
<a href="https://aws.amazon.com/sns/" target="_bank">AWS</a>
</p>

    




### Creating a client

<p>
"Clients provide a low-level interface to AWS whose methods map close to 1:1 with service APIs. All service operations are supported by clients. Clients are generated from a JSON service definition file." (Boto3 doc)
</p>

<p>
The name of the service and the keys are required.<br>
The service name here is 'sns'.<br>
</p>

In [17]:
s3_sns_client = boto3.client('sns', region_name='us-east-1',
                  aws_access_key_id=Access_Key_ID,
                  aws_secret_access_key=Secret_Access_Key)

In [18]:
s3_sns_client

#### Create topics

<p>
"An Amazon SNS topic is a logical access point that acts as a communication channel.<br>
A topic lets you group multiple endpoints (such as AWS Lambda, Amazon SQS, HTTP/S, or an email address)."
</p>

In [19]:
tornado_reponse = \
s3_sns_client.create_topic(Name='tornado_notification_service')

In [20]:
taifun_response = \
s3_sns_client.create_topic(Name="taifun_notification_service")  

In [21]:
# tornado_response['TopicArn']
tornado_topic_arn = tornado_reponse['TopicArn']
print(tornado_topic_arn)

taifun_topic_arn = taifun_response['TopicArn']
print(taifun_topic_arn)

arn:aws:sns:us-east-1:047675893846:tornado_notification_service
arn:aws:sns:us-east-1:047675893846:taifun_notification_service


In [22]:
topics_response_1 = s3_sns_client.list_topics()

print(topics_response_1['Topics'])

[{'TopicArn': 'arn:aws:sns:us-east-1:047675893846:Default_CloudWatch_Alarms_Topic'}, {'TopicArn': 'arn:aws:sns:us-east-1:047675893846:alarm_greater_than_one_dollar'}, {'TopicArn': 'arn:aws:sns:us-east-1:047675893846:taifun_notification_service'}, {'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}, {'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system'}]


### Amazon Resource Names (ARNs)

<p>
"Amazon Resource Names (ARNs) uniquely identify AWS resources.<br>
We require an ARN when you need to specify a resource unambiguously across all of AWS,<br>
such as in IAM policies, Amazon Relational Database Service (Amazon RDS) tags, and API calls."
</p>

#### Which ARNs are currently active?

In [75]:
def get_topic_arn_list(client):
        topic_arn_list = []

        list_topics_22 = client.list_topics()['Topics']

        for i in list_topics_22:
            topic_arn = i['TopicArn'].split(':')[-1]
            topic_arn_list.append(topic_arn)
            
        return topic_arn_list 


In [73]:
topic_arn_list_2 =\
get_topic_arn_list(client = s3_sns_client)

print(topic_arn_list_2)

['Default_CloudWatch_Alarms_Topic', 'alarm_greater_than_one_dollar', 'tornado_notification_service', 'tornado_notification_service_system']


#### Delete ARNs

<p>For deletion the ARN adress is needed.</p>

In [24]:
taifun_deletion = \
s3_sns_client.delete_topic(TopicArn=taifun_topic_arn)

#### Is the taifun_notification_service still there?

In [74]:
topic_arn_list_2 =\
get_topic_arn_list(client = s3_sns_client)

print(topic_arn_list_2)
    

['Default_CloudWatch_Alarms_Topic', 'alarm_greater_than_one_dollar', 'tornado_notification_service', 'tornado_notification_service_system']


#### Create subscriptions
<p>
for topics.<br>
In this case, subscribe to the tornado notification service.
</p>

In [26]:
subscription_1 = \
s3_sns_client.subscribe(
TopicArn = tornado_topic_arn,
Protocol = 'email',
Endpoint = 'iomer3423@mailfence.com')

In [27]:
subscription_2 = \
s3_sns_client.subscribe(
TopicArn = tornado_topic_arn,
Protocol = 'email',
Endpoint = 'Gotermer3534@yahoo.com')

In [28]:
subscription_3 = \
s3_sns_client.subscribe(
TopicArn = tornado_topic_arn,
Protocol = 'email',
Endpoint = 'ermerdir.itoplasto@aol.com')

In [29]:
print(subscription_1['SubscriptionArn'])
print(subscription_2['SubscriptionArn'])
print(subscription_3['SubscriptionArn'])

pending confirmation
pending confirmation
pending confirmation


## Waiting for confirmation
<p>
After confirmation by the recipient.
</p>

In [30]:
print(subscription_1)

{'SubscriptionArn': 'pending confirmation', 'ResponseMetadata': {'RequestId': '5fd9e8ee-5dfc-50dd-83a1-b315bd6fd22b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '5fd9e8ee-5dfc-50dd-83a1-b315bd6fd22b', 'content-type': 'text/xml', 'content-length': '298', 'date': 'Sat, 26 Nov 2022 09:37:22 GMT'}, 'RetryAttempts': 0}}


#### Which subscriptions are existing?

In [31]:
all_subscriptions = \
s3_sns_client.list_subscriptions()['Subscriptions']
print(all_subscriptions)

[{'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:db337d9d-8def-4065-b8ff-c1919885d3dc', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'iomer3423@mailfence.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}, {'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system:e38d5980-8a4a-4ff8-a867-dfb54106df67', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'ermerdir.itoplasto@aol.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system'}, {'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system:4fff18ee-b3b3-479a-a311-64ea04ae8f83', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'Gotermer3534@yahoo.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system'}, {'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-474

In [33]:
# getting only the endpoint and the arn's

print(len(all_subscriptions))

confirmed_1 = {}

for i in range(len(all_subscriptions)):
    enpoint_22 = s3_sns_client.list_subscriptions()['Subscriptions'][i]['Endpoint']
    arn_22 = s3_sns_client.list_subscriptions()['Subscriptions'][i]['SubscriptionArn']
    confirmed_1[enpoint_22] = arn_22

confirmed_1

4


{'iomer3423@mailfence.com': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:db337d9d-8def-4065-b8ff-c1919885d3dc',
 'ermerdir.itoplasto@aol.com': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service_system:e38d5980-8a4a-4ff8-a867-dfb54106df67',
 'Gotermer3534@yahoo.com': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-4745-9ee0-6bde12ca2cd5'}

#### Which emails are subscribed to the tornado topic?

In [34]:
subscription_dict_tornado = \
s3_sns_client.list_subscriptions_by_topic(
TopicArn=tornado_topic_arn)

In [36]:
subscription_dict_tornado['Subscriptions'][0].keys()

dict_keys(['SubscriptionArn', 'Owner', 'Protocol', 'Endpoint', 'TopicArn'])

<p>
The 'SubscriptionArn' identifies the subscriber.
</p>

In [37]:
for i in range(0,2):
    print(subscription_dict_tornado['Subscriptions'][i]['Endpoint'])

ermerdir.itoplasto@aol.com
iomer3423@mailfence.com


In [38]:
subscription_dict_tornado['Subscriptions'][0]

{'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:2c0a6dd0-c9f4-40fd-9d60-01330b2e2c22',
 'Owner': '047675893846',
 'Protocol': 'email',
 'Endpoint': 'ermerdir.itoplasto@aol.com',
 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}

In [39]:
arn_itoplast_aol = subscription_dict_tornado['Subscriptions'][0]['SubscriptionArn']
arn_iomer_mailfence = subscription_dict_tornado['Subscriptions'][1]['SubscriptionArn']
arn_gotermer_yahoo = subscription_dict_tornado['Subscriptions'][2]['SubscriptionArn']

print(arn_itoplast_aol)
print(arn_iomer_mailfence)
print(arn_gotermer_yahoo)

arn:aws:sns:us-east-1:047675893846:tornado_notification_service:2c0a6dd0-c9f4-40fd-9d60-01330b2e2c22
arn:aws:sns:us-east-1:047675893846:tornado_notification_service:db337d9d-8def-4065-b8ff-c1919885d3dc
arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-4745-9ee0-6bde12ca2cd5


#### Filter for protocols (email, app, sms ...)

In [40]:
subs_222 = subscription_dict_tornado['Subscriptions']

for sub in subs_222:
    if sub['Protocol'] == "email":
        print(sub['Endpoint'])
        

ermerdir.itoplasto@aol.com
iomer3423@mailfence.com
Gotermer3534@yahoo.com


<p>
This user defined function could be added to the helper package.
</p>

In [41]:
def filter_protocol(topic_dict, protocol):
    for sub in topic_dict:
        if sub['Protocol'] == protocol:
            return sub['Endpoint']
            
filter_protocol(topic_dict=subs_222, protocol='email')

'ermerdir.itoplasto@aol.com'

#### Unsubscribe

In [42]:
s3_sns_client.unsubscribe(
    SubscriptionArn = arn_itoplast_aol)

{'ResponseMetadata': {'RequestId': '943616ff-05c4-5192-8d2c-4d04e75440f1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '943616ff-05c4-5192-8d2c-4d04e75440f1',
   'content-type': 'text/xml',
   'content-length': '201',
   'date': 'Sat, 26 Nov 2022 09:45:52 GMT'},
  'RetryAttempts': 0}}

In [43]:
if arn_iomer_mailfence.startswith('arn'):
    s3_sns_client.unsubscribe(SubscriptionArn = arn_iomer_mailfence)
else:
    print(arn_iomer_mailfence)

In [44]:
for i in range(0,2):
    print(subscription_dict_tornado['Subscriptions'][i]['Endpoint'])

ermerdir.itoplasto@aol.com
iomer3423@mailfence.com


#### Unsubscribe multiple endpoints by topic and protocol

<p>
This user defined function could be added to the helper package.
</p>

In [45]:
def unsubscribe_protocol(client, TopicArn, protocol):
    
    unsubscribed_SubscriptionArns = {}
    
    client_333 = eval(client)
    print(type(client_333))
    response_555 = client_333.list_subscriptions_by_topic(TopicArn=TopicArn)
    subs555 = response_555['Subscriptions']
    print(subs555)
    
    for sub in subs555:
        if (sub['SubscriptionArn']== 'PendingConfirmation'):
            continue
    
        if (sub['Protocol'] == protocol):
                print(sub['SubscriptionArn'])
                client_333.unsubscribe(SubscriptionArn=sub['SubscriptionArn'])
                print(protocol)
                
                unsubscribed_SubscriptionArns['Endpoint'] = sub['SubscriptionArn']
                
        return unsubscribed_SubscriptionArns
                

In [46]:
unsubscribed_SubscriptionArns_1 = \
unsubscribe_protocol(client='s3_sns_client', 
                     TopicArn=tornado_topic_arn, 
                     protocol='email')

unsubscribed_SubscriptionArns_1

<class 'botocore.client.SNS'>
[{'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-4745-9ee0-6bde12ca2cd5', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'Gotermer3534@yahoo.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}]
arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-4745-9ee0-6bde12ca2cd5
email


{'Endpoint': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:1e5039aa-b145-4745-9ee0-6bde12ca2cd5'}

#### Resubscribe

In [47]:
resubscribe_1 = \
["ermerdir.itoplasto@aol.com", "iomer3423@mailfence.com", "Gotermer3534@yahoo.com"]

resubscribe_1_responses = []

for i in resubscribe_1:
    
    response_1 = \
    s3_sns_client.subscribe(
    TopicArn = tornado_topic_arn,
    Protocol = 'email',
    Endpoint = i)
    
    resubscribe_1_responses.append(response_1)
print(resubscribe_1_responses[0] )

{'SubscriptionArn': 'pending confirmation', 'ResponseMetadata': {'RequestId': '73905c68-f424-530f-89ea-079316891425', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '73905c68-f424-530f-89ea-079316891425', 'content-type': 'text/xml', 'content-length': '298', 'date': 'Sat, 26 Nov 2022 09:46:21 GMT'}, 'RetryAttempts': 0}}


### Waiting for confirmation!

In [48]:
subs_899 =  s3_sns_client.list_subscriptions()['Subscriptions']

for i in subs_899:
    print(i['Endpoint'])

iomer3423@mailfence.com
ermerdir.itoplasto@aol.com
Gotermer3534@yahoo.com
ermerdir.itoplasto@aol.com


### Publishing notification messages

<p>
That is what SNS is all about.
</p>

In [49]:
publish_response_1 = \
s3_sns_client.publish(
    TopicArn=tornado_topic_arn,
    Message = "The tornado 'Chelsea' is coming tomorrow between 3 PM and 5 PM. \n It orignated in the West Atlantic. \n The pace is going to be around 510 KM / h. (F5). \n More news will follow.",
    Subject = "Breaking news: a new tornado is on the way!")

In [50]:
publish_response_1 


{'MessageId': '50a624e8-5b47-57ed-9c58-7f13b9a9c6b1',
 'ResponseMetadata': {'RequestId': 'ae77b882-bed3-5650-ab56-e3795c3d070c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ae77b882-bed3-5650-ab56-e3795c3d070c',
   'content-type': 'text/xml',
   'content-length': '294',
   'date': 'Sat, 26 Nov 2022 09:47:26 GMT'},
  'RetryAttempts': 0}}

#### Unsuscribe
<p>
all suscribers.<br>
Making a clean slate on AWS for the next project:<br>
<b>"Setting up a tornado warning system with AWS Simple Notification Service (SNS)"</b>
</p>

In [51]:
unsubscribed_SubscriptionArns_2 = \
unsubscribe_protocol(client='s3_sns_client', 
                     TopicArn=tornado_topic_arn, 
                     protocol='email')


<class 'botocore.client.SNS'>
[{'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:23ca6abf-79fa-4c2a-a364-2a3ab95a0425', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'Gotermer3534@yahoo.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}, {'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:0232d62c-4e61-4010-8116-78b1bef4adbe', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'iomer3423@mailfence.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}, {'SubscriptionArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:7cf9d0f2-77e5-404d-879b-eed03502b2dd', 'Owner': '047675893846', 'Protocol': 'email', 'Endpoint': 'ermerdir.itoplasto@aol.com', 'TopicArn': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service'}]
arn:aws:sns:us-east-1:047675893846:tornado_notification_service:23ca6abf-79fa-4c2a-a364-2a3ab95a0425

In [52]:
unsubscribed_SubscriptionArns_2

{'Endpoint': 'arn:aws:sns:us-east-1:047675893846:tornado_notification_service:23ca6abf-79fa-4c2a-a364-2a3ab95a0425'}

In [53]:
subs_111 =  s3_sns_client.list_subscriptions()['Subscriptions']

for i in subs_111:
    print(i['Endpoint'])

iomer3423@mailfence.com
ermerdir.itoplasto@aol.com
Gotermer3534@yahoo.com
ermerdir.itoplasto@aol.com
